In [12]:
import sys
import os
import yaml
from dataclasses import asdict

sys.path.append("./CLAP")
current_dir = os.getcwd()

In [13]:
mo833_nome_do_usuario = "mo833-2022-matheus" # Modifique esta variável com o seu usuário de nuvem mo833-...

In [14]:
from app.cli.modules.node import get_config_db, get_node_manager
from app.cli.modules.role import get_role_manager
from app.cli.modules.cluster import get_cluster_config_db, get_cluster_manager
from clap.utils import float_time_to_string

In [15]:
instance_config_db = get_config_db()
cluster_config_db = get_cluster_config_db()
node_manager = get_node_manager()
role_manager = get_role_manager()
cluster_manager = get_cluster_manager()
private_path = node_manager.private_path

In [16]:
t2_medium_type = instance_config_db.instance_descriptors["type-t2.medium"]
asdict(t2_medium_type)

{'provider': {'provider_config_id': 'aws-config-us-east-1',
  'region': 'us-east-1',
  'access_keyfile': 'ec2_access_key.pub',
  'secret_access_keyfile': 'ec2_access_key.pem',
  'vpc': None,
  'url': None,
  'provider': 'aws'},
 'login': {'login_config_id': 'login-ubuntu',
  'user': 'ubuntu',
  'keypair_name': 'mo833-matheus-lindino',
  'keypair_public_file': 'key.pub',
  'keypair_private_file': 'key.pem',
  'ssh_port': 22,
  'sudo': True,
  'sudo_user': 'root'},
 'instance': {'instance_config_id': 'type-t2.medium',
  'provider': 'aws-config-us-east-1',
  'login': 'login-ubuntu',
  'flavor': 't2.medium',
  'image_id': 'ami-0c4f7023847b90238',
  'security_group': 'open-security-group',
  'boot_disk_size': 16,
  'boot_disk_device': None,
  'boot_disk_type': None,
  'boot_disk_iops': None,
  'boot_disk_snapshot': None,
  'placement_group': None,
  'price': None,
  'timeout': None,
  'network_ids': []}}

In [17]:
node_ids = node_manager.start_node(t2_medium_type, count=2)
print(f"Nodes {', '.join(node_ids)} were started!")

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'
[DEPRECATION WARNING]: amazon.aws.ec2 has been deprecated. The ec2 module is 
based upon a deprecated version of the AWS SDKs and is deprecated in favor of 
the ec2_instance module. Please update your tasks. This feature will be removed
 from amazon.aws in version 4.0.0. Deprecation warnings can be disabled by 
setting deprecation_warnings=False in ansible.cfg.

PLAY [localhost] ***************************************************************

TASK [Starting 2 type-t2.medium instances (timeout 600 seconds)] ***************
deprecation_warnings=False in ansible.cfg.
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
[WARNING]: provided hosts list is empty, only localhost is available. Note that
the 

In [18]:
alive_nodes = node_manager.is_alive(node_ids)
for node_id, alive_flag in alive_nodes.items():
    alive_str = 'alive' if alive_flag == True else 'not alive'
    print(f"{node_id} --> {alive_str}.")

Error executing command in node c1dafb55: [Errno None] Unable to connect to port 22 on 3.83.239.108
Error executing command in c1dafb55: [Errno None] Unable to connect to port 22 on 3.83.239.108.


8fd30b25f00c4338ba4971f4b6d32eb6 --> alive.
c1dafb5565964930a78c03206b83adc8 --> alive.


In [19]:
# Variaveis espeficica por nó
node_vars={
    node_ids[0]: {
        "node_rank": 0
    }, 
    node_ids[1]: {
        "node_rank": 1
    }
}

# Variaveis passadas para a ação setup
extra_args = {
    "efs_host": "172.31.15.9",
    "efs_mount_point": "/home/ubuntu/efs/",
    "user_dir": f"/home/ubuntu/efs/{mo833_nome_do_usuario}/"
}


nodes_added_to_role = role_manager.add_role(
    role_name="dcgan", 
    hosts_node_map=node_ids,
    node_vars=node_vars,
    extra_args=extra_args,
)

print(f"O papel dcgan foram adicionados aos nós {', '.join(nodes_added_to_role)}")


PLAY [Acao de setup] ***********************************************************

TASK [Gathering Facts] *********************************************************
ok: [c1dafb5565964930a78c03206b83adc8]
ok: [8fd30b25f00c4338ba4971f4b6d32eb6]

TASK [Task 1: Lista as variáveis recebidas] ************************************
ok: [8fd30b25f00c4338ba4971f4b6d32eb6] => {
    "msg": "Variaveis: efs_host: 172.31.15.9; efs_mount_point: /home/ubuntu/efs/; user_dir: /home/ubuntu/efs/mo833-2022-matheus/"
}
ok: [c1dafb5565964930a78c03206b83adc8] => {
    "msg": "Variaveis: efs_host: 172.31.15.9; efs_mount_point: /home/ubuntu/efs/; user_dir: /home/ubuntu/efs/mo833-2022-matheus/"
}

TASK [Task 2: Instala dependencias] ********************************************
changed: [c1dafb5565964930a78c03206b83adc8]
changed: [8fd30b25f00c4338ba4971f4b6d32eb6]

TASK [Task 3: Cria o grupo docker] *********************************************
ok: [8fd30b25f00c4338ba4971f4b6d32eb6]
ok: [c1dafb5565964930a78c03206b83

In [20]:
# Variaveis espeficica por nó
node_vars={
    node_ids[0]: {
        "node_rank": 0
    }, 
    node_ids[1]: {
        "node_rank": 1
    }
}

# Variaveis passadas para a ação setup
extra_args = {
    "efs_mount_point": "/home/ubuntu/efs/",
    "user_dir": f"/home/ubuntu/efs/{mo833_nome_do_usuario}/",
    "cifar_data_dir": "/home/ubuntu/efs/data/cifar10/",
    "master_hostname": node_ids[0],
    "results_dir": f"/home/ubuntu/efs/{mo833_nome_do_usuario}/Distributed-DCGAN/resultados/",
}

res = role_manager.perform_action(
    role_name="dcgan", 
    action_name="run", 
    hosts_node_map=node_ids, 
    node_vars=node_vars,
    extra_args=extra_args
)

print(f"Playbook executou bem? {res.ok}")


PLAY [Run Play] ****************************************************************

TASK [Task 1: Lista as variáveis recebidas] ************************************
ok: [8fd30b25f00c4338ba4971f4b6d32eb6] => {
    "msg": "Variaveis: efs_mount_point: /home/ubuntu/efs/; user_dir: /home/ubuntu/efs/mo833-2022-matheus/; cifar_data_dir: /home/ubuntu/efs/data/cifar10/; master_hostname: 8fd30b25f00c4338ba4971f4b6d32eb6; results_dir: /home/ubuntu/efs/mo833-2022-matheus/Distributed-DCGAN/resultados/;"
}
ok: [c1dafb5565964930a78c03206b83adc8] => {
    "msg": "Variaveis: efs_mount_point: /home/ubuntu/efs/; user_dir: /home/ubuntu/efs/mo833-2022-matheus/; cifar_data_dir: /home/ubuntu/efs/data/cifar10/; master_hostname: 8fd30b25f00c4338ba4971f4b6d32eb6; results_dir: /home/ubuntu/efs/mo833-2022-matheus/Distributed-DCGAN/resultados/;"
}

TASK [Task 2: Criar diretório de resultados] ***********************************
ok: [8fd30b25f00c4338ba4971f4b6d32eb6]

TASK [Task 3: Executar a DCGAN] ****************

In [21]:
# Variaveis espeficica por nó
node_vars={
    node_ids[0]: {
        "node_rank": 0
    }, 
    node_ids[1]: {
        "node_rank": 1
    }
}

# Variaveis passadas para a ação setup
extra_args = {
    "user_dir": f"/home/ubuntu/efs/{mo833_nome_do_usuario}/",
    "results_dir": f"{current_dir}/resultados",
    "results_remote": f"/home/ubuntu/efs/{mo833_nome_do_usuario}/Distributed-DCGAN/resultados/"
}


res = role_manager.perform_action(
    role_name="dcgan", 
    action_name="results", 
    hosts_node_map=node_ids, 
    node_vars=node_vars,
    extra_args=extra_args,
    quiet=False
)

print(f"Playbook executou bem? {res.ok}")


PLAY [Acao de resultados] ******************************************************

TASK [Gathering Facts] *********************************************************
ok: [c1dafb5565964930a78c03206b83adc8]
ok: [8fd30b25f00c4338ba4971f4b6d32eb6]

TASK [Task 1: Lista as variáveis recebidas] ************************************
ok: [8fd30b25f00c4338ba4971f4b6d32eb6] => {
    "msg": "Variaveis: results_dir: /home/lindino/mo833-atividade9/resultados; user_dir: /home/ubuntu/efs/mo833-2022-matheus/; results_remote: /home/ubuntu/efs/mo833-2022-matheus/Distributed-DCGAN/resultados/"
}
ok: [c1dafb5565964930a78c03206b83adc8] => {
    "msg": "Variaveis: results_dir: /home/lindino/mo833-atividade9/resultados; user_dir: /home/ubuntu/efs/mo833-2022-matheus/; results_remote: /home/ubuntu/efs/mo833-2022-matheus/Distributed-DCGAN/resultados/"
}

TASK [Task 2: Copia os arquivos output-*.txt] **********************************
changed: [c1dafb5565964930a78c03206b83adc8]
changed: [8fd30b25f00c4338ba4971f4b6d3

In [22]:
stopped_nodes = node_manager.stop_nodes(node_ids)
print(f"Stopped nodes: {', '.join(stopped_nodes)}")

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'
[DEPRECATION WARNING]: amazon.aws.ec2 has been deprecated. The ec2 module is 
based upon a deprecated version of the AWS SDKs and is deprecated in favor of 
the ec2_instance module. Please update your tasks. This feature will be removed
 from amazon.aws in version 4.0.0. Deprecation warnings can be disabled by 
setting deprecation_warnings=False in ansible.cfg.

PLAY [localhost] ***************************************************************

TASK [Stopping nodes AnneRaymond, SherriKelly] *********************************
[DEPRECATION WARNING]: The 'ec2' module has been deprecated and replaced by the
 'ec2_instance' module'. This feature will be removed from amazon.aws in 
version 4.0.0. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.
changed: [localhost]

PLAY RECAP ******************************************************